In [1]:
from bw2data import Database, Method
import bw2data as bd
import brightway2 as bw
import bw2data
from tqdm import tqdm  # Progress bar
from bw2data import Database
from premise.geomap import Geomap
import wurst as w
import uuid
from tqdm import tqdm
from regionalization import match_best_location, relink_technosphere_exchanges

Adding REMIND and IMAGE topology using regionalization.py


In [2]:
from db_functions import * # this is a python file with self-created functions needed to run this notebook
from config import * #some main key parameters needed.
from private_keys import * #import premise key
from functions import find_activity_fast

In [3]:
bw.projects.set_current(PROJECT_NAME) #Creating/accessing the project
DB_NAME_INIT,PROJECT_NAME

('ecoinvent-3.10-cutoff', 'lca_steel_sector_cbam')

In [4]:
all_ei_databases = [db_2 for db_2 in bw.databases if "ecoinvent_" in str(db_2)]
all_ei_databases

['ecoinvent_310_reference',
 'ecoinvent_remind_SSP2-PkBudg1150_2030_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2035_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2045_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2050_base',
 'ecoinvent_image_SSP2-RCP26_2030_base',
 'ecoinvent_image_SSP2-RCP26_2035_base',
 'ecoinvent_image_SSP2-RCP26_2040_base',
 'ecoinvent_image_SSP2-RCP26_2045_base',
 'ecoinvent_image_SSP2-RCP26_2050_base',
 'ecoinvent_remind_SSP2-PkBudg1150_2040_base']

In [5]:
for db in list(bw.databases):
    if 'steel_db_remind_SSP2-PkBudg1150' in str(db):
        print(db)
        #del bw.databases[db]

### Make new databases going from IMAGE to REMIND

In [6]:
def get_activity_key(db_name, name, reference_product, location):
    """
    Retrieve the (database, code) key for an activity from a Wurst-style DB based on name, product, and location.

    Parameters:
    - db_name (str): Name of the database
    - db_sel (list or dict): Wurst-compatible database (usually a list of dicts)
    - name (str): Activity name
    - reference_product (str): Reference product
    - location (str): Location

    Returns:
    - tuple: (db_name, activity_code)
    """
    w_db = database_sel[db_name]
    act = w.get_one(
        w_db,
        w.equals("name", name),
        w.equals("reference product", reference_product),
        w.equals("location", location)
    )
    return (db_name, act["code"])

In [7]:
def write_new_db_with_new_iam_regions_full_relinking(
    w_db_to_modify,
    new_db_name,
    dict_db_dependencies,
    database_sel,
    db_sourced,
    get_prod_value=True,
    regionalize=True, 
    regionalize_exemption = ['fossil fuel']
):
    """
    Create a new Brightway2 database with updated regional information for IAM integration.

    This function duplicates a list of Brightway2 datasets, updates their database name and code,
    and relinks technosphere exchanges based on new IAM-specific regional mappings.

    Parameters:
    - w_db_to_modify: List of Brightway2 datasets (dicts) to be modified.
    - new_db_name: String name for the new Brightway2 database to be created.
    - dict_db_dependencies: Dictionary mapping old database names to their corresponding new IAM-specific database names.
    - database_sel: Dictionary mapping database names to Brightway2 `Database` objects used for relinking.

    Returns:
    - List of updated dataset dictionaries, ready to be written as a new Brightway2 database.
    """

    # Remove existing database if it exists
    if new_db_name in list(bw.databases):
        print("Deleting existing database...")
        del bw.databases[new_db_name]

    new_datasets = []

    for i, act in enumerate(tqdm(w_db_to_modify, desc="Updating datasets")):
        act = act.copy()
        act["database"] = new_db_name
        new_code = str(uuid.uuid4().hex)
        act["code"] = new_code

        # Now, we need to know the initial database it is linked to find the new database it should be linked to
        # note: this found database should be the same for all technosphere exchanges therefore we can stop after finding first instance
        init_db = None

        if get_prod_value:
            prod_vol = find_activity_fast(db_sourced, act['name'], act['reference product'], act['location'] )['production volume']
        
        new_prod = (new_db_name, new_code)
        
        for e in act["exchanges"]:
            if e["type"] == "technosphere" and init_db == None:
                init_db = e["input"][0]
                break
                
        # We need to know to which db to reink to.
        new_db_for_act = dict_db_dependencies[init_db] 

        act = act.copy()
        if regionalize and act['reference product'] not in regionalize_exemption:
            act = relink_technosphere_exchanges(act, database_sel[new_db_for_act], contained=False)

        exchanges = []
        for exc in act['exchanges']:
            exc = exc.copy()
            if exc['type'] == 'technosphere':
                product = exc.get("product") or exc.get("reference product")
                #print(new_db_for_act, exc["name"], exc["location"])
                exc["input"] = get_activity_key(
                                db_name=new_db_for_act,
                                name=exc["name"],
                                location=exc["location"],
                                reference_product=product
                            )
                    
                exchanges.append(exc)
            elif exc['type'] == 'production':
                exc["input"] = (new_db_name, new_code)                    
                exchanges.append(exc)
            else:
                # no relinking needed for biosphere flows
                exchanges.append(exc)
                
        if prod_vol is None and get_prod_value:
            print(f"WARNING: no production volume found for: {act['name']}")

        act['exchanges'] = exchanges
        
        if get_prod_value:
            act['production volume'] = prod_vol
            
        new_datasets.append(act)

    return new_datasets

In [8]:
new_iam_model = 'remind'
new_iam_model_scenario = 'SSP2-PkBudg1150'

old_iam_model = 'image'
old_iam_model_scenario = 'SSP2-RCP26'

new_iam_scenario = f'{new_iam_model}_{new_iam_model_scenario}'
old_iam_scenario = f'{old_iam_model}_{old_iam_model_scenario}'

# extract the databases using wurst
ref_db_w = w.extract_brightway2_databases(NAME_REF_DB, add_properties=False, add_identifiers=False)
ref_db_w_2030 = w.extract_brightway2_databases(f'ecoinvent_{new_iam_scenario}_2030_base', add_properties=False, add_identifiers=False)
ref_db_w_2035 = w.extract_brightway2_databases(f'ecoinvent_{new_iam_scenario}_2035_base', add_properties=False, add_identifiers=False)
ref_db_w_2040 = w.extract_brightway2_databases(f'ecoinvent_{new_iam_scenario}_2040_base', add_properties=False, add_identifiers=False)
ref_db_w_2045 = w.extract_brightway2_databases(f'ecoinvent_{new_iam_scenario}_2045_base', add_properties=False, add_identifiers=False)
ref_db_w_2050 = w.extract_brightway2_databases(f'ecoinvent_{new_iam_scenario}_2050_base', add_properties=False, add_identifiers=False)
bio_db_w = w.extract_brightway2_databases(BIOSPHERE_DB, add_properties=False, add_identifiers=False)

# pre-load the databases
database_sel = {NAME_REF_DB:ref_db_w,
                BIOSPHERE_DB:bio_db_w,
                f'ecoinvent_{new_iam_scenario}_2030_base':ref_db_w_2030,
                f'ecoinvent_{new_iam_scenario}_2035_base':ref_db_w_2035,
                f'ecoinvent_{new_iam_scenario}_2040_base':ref_db_w_2040,
                f'ecoinvent_{new_iam_scenario}_2045_base':ref_db_w_2045,
                f'ecoinvent_{new_iam_scenario}_2050_base':ref_db_w_2050,                
               }

# for new linking
dict_db_dependencies = {
    BIOSPHERE_DB: BIOSPHERE_DB,
    NAME_REF_DB: NAME_REF_DB,
    f'ecoinvent_{old_iam_scenario}_2030_base': f'ecoinvent_{new_iam_scenario}_2030_base',
    f'ecoinvent_{old_iam_scenario}_2035_base': f'ecoinvent_{new_iam_scenario}_2035_base',
    f'ecoinvent_{old_iam_scenario}_2040_base': f'ecoinvent_{new_iam_scenario}_2040_base',
    f'ecoinvent_{old_iam_scenario}_2045_base': f'ecoinvent_{new_iam_scenario}_2045_base',
    f'ecoinvent_{old_iam_scenario}_2050_base': f'ecoinvent_{new_iam_scenario}_2050_base',
}

Getting activity data


100%|█████████████████████████████████████████████████████████████████████████| 26110/26110 [00:00<00:00, 33235.30it/s]


Adding exchange data to activities


100%|████████████████████████████████████████████████████████████████████████| 804790/804790 [01:39<00:00, 8097.61it/s]


Filling out exchange data


...

100%|███████████████████████████████████████████████████████████████████████████| 4362/4362 [00:00<00:00, 86723.17it/s]


Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|██████████████████████████████████████████████████████████████████████████| 4362/4362 [00:00<00:00, 872380.03it/s]


In [9]:
dbs_to_modify = ["steel_db_future", "steel_db_future_ccs"]

for db_to_modify in dbs_to_modify:
    w_db_to_modify = w.extract_brightway2_databases(db_to_modify, add_properties=False, add_identifiers=False)
    new_db_name = f"{db_to_modify}_{new_iam_scenario}"

    # write_new_db_with_new_iam_regions_full_relinking --> 20 mins, but links to all best technosphere locations within all activities
    new_datasets = write_new_db_with_new_iam_regions_full_relinking(
                            w_db_to_modify,
                            new_db_name,
                            dict_db_dependencies,
                            database_sel,
                            db_sourced = db_to_modify
                        )

    process_import(new_db_name, new_datasets, iam=new_iam_model)

Getting activity data


100%|████████████████████████████████████████████████████████████████████████████| 711/711 [00:00<00:00, 142183.19it/s]


Adding exchange data to activities


100%|██████████████████████████████████████████████████████████████████████████| 57105/57105 [00:09<00:00, 5888.27it/s]


Filling out exchange data


...

Deleting existing database...
Vacuuming database 


Updating datasets: 100%|█████████████████████████████████████████████████████████████| 711/711 [39:50<00:00,  3.36s/it]


Applying strategy: add_database_name
Applying strategy: csv_restore_tuples
Applying strategy: drop_empty_categories
Applying strategy: drop_empty_categories_2
Applying strategy: strip_nonsense
Applied 5 strategies in 3.87 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
711 datasets
71885 exchanges
0 unlinked exchanges
  
Wrote matching file to:
C:\Users\terlouw_t\AppData\Local\pylca\Brightway3\lca_steel_sector_cbam.27d78f50af11f772309a5a15dc8c3210\output\db-matching-steel_db_future_ccs_remind_SSP2-PkBudg1150-unlinked.xlsx


Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Title: Writing activities to SQLite3 database:
  Started: 07/23/2025 11:21:20
  Finished: 07/23/2025 11:21:30
  Total time elapsed: 00:00:10
  CPU %: 65.80
  Memory %: 24.18
Created database: steel_db_future_ccs_remind_SSP2-PkBudg1150
